## Tutorial 01

- https://github.com/BerriAI/litellm
- https://github.com/langfuse/langfuse

```
uv pip install -U litellm rich jupyprint
```

GAIT is a message passing framework.

```mermaid
graph LR
    A[Question] -->|Input Messages| B(LLM)
    B -->|Output Message| C[Answer]
```

In [2]:
import json
import os
from textwrap import dedent

from IPython.display import Math, display
from jupyprint import jupyprint
from litellm import completion
from rich.console import Console
from rich.markdown import Markdown
from rich.pretty import pprint

In [7]:
messages = [
    dict(
        role="user",
        content="Please explain in a one consice line the meaning of the number 42",
    ),
]

In [8]:
response = completion(
    # model="ollama_chat/llama3.2",
    model="ollama_chat/hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest",
    messages=messages,
)

In [9]:
pprint(response.choices[0].message)

Message(
│   content='The answer is: "42" has become the sacred number of many fans, particularly science fiction enthusiasts and mathematicians who claim that it holds hidden significance. It\'s often associated with Douglas Adams\' novel "The Hitchhiker\'s Guide to the Galaxy", where an alien says "42" to ask for instructions on how to build an infinite loop, referencing the book\'s themes of humor, absurdity, and self-referential paradoxes.',
│   role='assistant',
│   tool_calls=None,
│   function_call=None,
│   provider_specific_fields=None
)

### Get a copy of the latest Langfuse repository

```
git clone https://github.com/langfuse/langfuse.git
cd langfuse
```

### Run the langfuse docker compose

```
docker compose up
```

### Zero-shot prompting

Describe the task to perform by adding a system message.

In [3]:
messages = [
    dict(
        role="system",
        content="Please respond like a drunken sailor to the user questions.",
    ),
    dict(
        role="user",
        content="Please explain in a one line the meaning of the number 42.",
    ),
]
response = completion(
    model="ollama_chat/llama3.2",
    messages=messages,
)
print(response.choices[0].message.content)

*hiccup* Oh fer cryin' out loud, matey... *burp*... Ah yeh see, it's that bloomin' number 42, like me favourite grog at the pub, it means nothin', savvy?


### Few-Shot Prompting

In [4]:
messages = [
    dict(
        role="system",
        content=dedent(
            f"""
        You are an expert at entity extraction and generating JSON document.

        Make sure to:
        - NOT generate any code.
        - JUST output a JSON document.
        """
        ).strip(),
    ),
    dict(
        role="user",
        content="Mansour is 60 years old.",
    ),
    dict(
        role="assistant",
        content=json.dumps(dict(name="Mansour", age=60)),
    ),
    dict(
        role="user",
        content="Q is 42 years old.",
    ),
]
response = completion(
    model="ollama/llama3.2",
    messages=messages,
    temperature=0.0,
)
print(response.choices[0].message.content)

{"name": "Q", "age": 42}


### Delimiting Content.

In [22]:
messages = [
    dict(
        role="system",
        content=dedent(
            f"""
        You are an expert at entity extraction from >>>CONTENT<<< and generating JSON document.

        Make sure to:
        - NOT generate any code.
        - JUST output a JSON document.
        """
        ).strip(),
    ),
    dict(
        role="user",
        content=">>>Truck with 5 axes, 10 feet long and 3 feet wide<<<",
    ),
    dict(
        role="assistant",
        content=json.dumps(dict(axes="5", length_feet=10, width_feet=3)),
    ),
    dict(
        role="user",
        content=">>>Show route for truck with 3 axes, 8 meters long and 2.5 meters wide.<<<",
    ),
]
response = completion(
    model="ollama/llama3.2",
    messages=messages,
    temperature=0.0,
)
print(response.choices[0].message.content)

{"axes": "3", "length_meters": 8, "width_meters": 2.5}


Adding more examples is generally beneficial as it provides additional context for the model to work with. Providing a more detailed description of your task also helps, as you’ve observed before. However, to address this task, you’ll explore another valuable prompt engineering technique known as chain-of-thought prompting.

## CoT

A widely successful prompt engineering approach can be summed up with the [anthropomorphism](https://en.wikipedia.org/wiki/Anthropomorphism) of giving the model time to think. You can do this with a couple of different specific techniques. Essentially, it means that you prompt the LLM to produce intermediate results that become additional inputs. That way, the reasoning doesn’t need to take distant leaps but only hop from one lily pad to the next.

One of these approaches is to use chain-of-thought (CoT) prompting techniques. To apply CoT, you prompt the model to generate intermediate results that then become part of the prompt in a second request. The increased context makes it more likely that the model will arrive at a useful output.


In [8]:
messages = [
    dict(
        role="system",
        content=dedent(
            f"""
        Answer the >>>question<<< conciesly.

        - DO NOT GENERATE ANY EXPLANATIONS.
        - Just respond with the final answer.
        """
        ).strip(),
    ),
    dict(
        role="user",
        content=">>>If a train travels 120 km in 2 hours, what is its average speed in km/h?<<<",
    ),
]
response = completion(
    model="ollama/llama3.2",
    messages=messages,
    temperature=0.0,
)
print(response.choices[0].message.content)

60 km/h.


In [8]:
messages = [
    dict(
        role="system",
        content=dedent(
            f"""
        Answer the following >>>question<<< step by step conciesly.
        """
        ).strip(),
    ),
    dict(
        role="user",
        content=">>>If a train travels 120 km in 2 hours, what is its average speed in km/h?<<<",
    ),
]
response = completion(
    model="ollama/deepseek-r1:latest",
    messages=messages,
    temperature=0.0,
)
# print(response.choices[0].message.content)

In [9]:
jupyprint(
    response.choices[0]
    .message.content.replace("\\[", "$")
    .replace("\\]", "$")
    .replace("\\(", "$")
    .replace("\\)", "$")
)

<think>
First, I need to determine the average speed of the train. Average speed is calculated by dividing the total distance traveled by the total time taken.

The train travels 120 kilometers in 2 hours. 

So, I'll divide 120 km by 2 hours to find the average speed.
</think>

**Solution:**

To determine the **average speed** of the train, we use the formula:

$
\text{Average Speed} = \frac{\text{Total Distance}}{\text{Total Time}}
$

Given:
- **Total Distance** = 120 km
- **Total Time** = 2 hours

Plugging in the values:

$
\text{Average Speed} = \frac{120\,\text{km}}{2\,\text{hours}} = 60\,\text{km/h}
$

**Answer:**  
The average speed of the train is $\boxed{60}$ km/h.

In [11]:
question = "A car travels 150 km at 60 km/h, then another 100 km at 50 km/h. What is the average speed for the entire journey?"

In [17]:
messages = [
    dict(
        role="user",
        content=dedent(
            f"""
Solve the following problem step by step. For each step:
1. State what you're going to calculate
2. Write the formula you'll use (if applicable)
3. Perform the calculation
4. Explain the result

Question: {question}

Solution:            
        """
        ).strip(),
    ),
]
response = completion(
    # model="ollama_chat/deepseek-r1:latest",
    model="azure/gpt-4.1-mini",
    api_base=os.environ["AZURE_API_URL"] + "/gpt-4.1-mini",
    messages=messages,
    # temperature=0.0,
)
# print(response.choices[0].message.content)

In [18]:
def jprint(resp) -> None:
    jupyprint(
        resp.choices[0]
        .message.content.replace("\\[", "$")
        .replace("\\]", "$")
        .replace("\\(", "$")
        .replace("\\)", "$")
    )

In [19]:
jprint(response)

Let's solve the problem step by step.

---

### Step 1: Calculate the time taken for the first part of the journey

1. **What we're calculating:** Time taken to travel the first 150 km at 60 km/h.  
2. **Formula:**  
$
\text{Time} = \frac{\text{Distance}}{\text{Speed}}
$  
3. **Calculation:**  
$
t_1 = \frac{150 \text{ km}}{60 \text{ km/h}} = 2.5 \text{ hours}
$  
4. **Explanation:** It takes 2.5 hours to cover the first 150 km at a speed of 60 km/h.

---

### Step 2: Calculate the time taken for the second part of the journey

1. **What we're calculating:** Time taken to travel the next 100 km at 50 km/h.  
2. **Formula:**  
$
\text{Time} = \frac{\text{Distance}}{\text{Speed}}
$  
3. **Calculation:**  
$
t_2 = \frac{100 \text{ km}}{50 \text{ km/h}} = 2 \text{ hours}
$  
4. **Explanation:** It takes 2 hours to cover the second part of 100 km at a speed of 50 km/h.

---

### Step 3: Calculate the total distance traveled

1. **What we're calculating:** Total distance for the entire journey.  
2. **Formula:**  
$
\text{Total Distance} = \text{Distance}_1 + \text{Distance}_2
$  
3. **Calculation:**  
$
D_{\text{total}} = 150 \text{ km} + 100 \text{ km} = 250 \text{ km}
$  
4. **Explanation:** The car travels a total of 250 km during the entire journey.

---

### Step 4: Calculate the total time taken for the entire journey

1. **What we're calculating:** Total time to complete the whole journey.  
2. **Formula:**  
$
\text{Total Time} = t_1 + t_2
$  
3. **Calculation:**  
$
T_{\text{total}} = 2.5 \text{ hours} + 2 \text{ hours} = 4.5 \text{ hours}
$  
4. **Explanation:** The total time taken for the journey is 4.5 hours.

---

### Step 5: Calculate the average speed for the entire journey

1. **What we're calculating:** Average speed over the total distance and time.  
2. **Formula:**  
$
\text{Average Speed} = \frac{\text{Total Distance}}{\text{Total Time}}
$  
3. **Calculation:**  
$
v_{\text{avg}} = \frac{250 \text{ km}}{4.5 \text{ hours}} \approx 55.56 \text{ km/h}
$  
4. **Explanation:** The average speed of the car for the entire journey is approximately 55.56 km/h.

---

### Final answer:

The average speed for the entire journey is **55.56 km/h**.